## Benchmarks con Pubtator

Para realizar los benchmarks con Pubtator, lo debemos separar en dos partes:

1. **Carga de datos**: Obtendremos los distintos datasets en formato PubTator y los enviaremos por API al endpoint que tenemos disponible, una vez hecho esto lo guardaremos en un JSON.
2. **Extracción de resultados**: Tendremos que esperar varios días, hasta que se hayan procesado nuestros textos, y con la api de `retrive` podamos obtener los resultados.

### Instalación de Librerías

In [ ]:
import requests as req
import time
import json
from typing import Dict, Union
import pandas as pd
from ner_benchmark_data_wrangler import NERBenchmarkDataWrangler
from datasets_utils import datasets_map
import numpy as np
from datasets_utils import parse_bcdr_to_format
from datasets import load_dataset
from utils import calculate_metrics, read_pubtator_content


### API de Pubtator

Para poder realizar los benchmarks con Pubtator, vamos a crear la clase `PubtatorAPI` que nos permitirá interactuar con la API de Pubtator.

In [ ]:
class PubtatorAPI:
    @staticmethod
    def submit_request(text: str) -> Union[str, None]:
        url = "https://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/RESTful/request.cgi"
        response = req.post(url, data={'text': text}, headers={'Content-Type': 'application/x-www-form-urlencoded'})

        if response.status_code == 200:
            session_number = response.json().get('id', '')

            return session_number
        else:
            print(f"Error: {response.status_code}")
            return None

    @staticmethod
    def get_results(session_number: str) -> Union[str, None]:
        base_url = "https://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/RESTful/retrieve.cgi"
        response = req.post(base_url, data={'id': session_number}, headers={'Content-Type': 'application/x-www-form-urlencoded'})

        if response.status_code == 200:
            return response.text
        else:
            return None

### Carga de Datos en la API

A continuación, vamos a cargar los datos en la API de Pubtator. Usaremos los datos de entrenamiento de cada uno de nuestros datasets, ya que son los que más texto contienen y al no tener que entrenar un modelo, no necesitamos los datos de validación y test.

Además nos guardaremos los `session_number` en un JSON para poder recuperarlos más adelante y poder consultar los resultados.

In [ ]:
import time
import json
from typing import Dict

CHUNKS_AMOUNT = 10

session_info: Dict[str, str] = {}

datasets = list(datasets_map.keys())

for dataset_name in datasets_map.keys():
    time.sleep(3)

    session_info[dataset_name] = {}

    wrangler = NERBenchmarkDataWrangler(dataset_name, datasets_map)

    data = wrangler.to_pubtator('train')

    if not data:
      continue

    chunk_data = np.array_split(np.array(data), CHUNKS_AMOUNT)

    for i, chunk in enumerate(chunk_data):
      text = '\n\n'.join(chunk.tolist())

      session_number = PubtatorAPI.submit_request(text)

      session_info[dataset_name][str(i)] = session_number

In [ ]:
with open('session_info.json', 'w') as json_file:
    json.dump(session_info, json_file)

In [ ]:
print(session_info)

json.dump(session_info, open('session_info.json', 'w'))

### Extracción de Resultados

Por último, vamos a extraer los resultados de la API Pubtator. Para ello, vamos a usar el endpoint de `retrive` y posteriormente haremos la evaluación de los resultados obtenidos.

**Nota**: La parte de evaluación no está realizada ya que todavía no he podido obtener los datos de pubtator.

In [ ]:
with open('session_info.json', 'r') as f:
  session_info = json.load(f)

In [ ]:
final_results = {}

for dataset_name, dataset_info in datasets_map.items():
    print(f"Trabajando con {dataset_name}")
    dataset = load_dataset(dataset_info['dataset_name'], split='train', trust_remote_code=True)
    dataset = dataset.to_pandas()

    ## Parseamos BC5DR ya que tiene un formato diferente
    if dataset_name == 'bc5dr':
        dataset = parse_bcdr_to_format(dataset, entity_map)
        dataset.rename(columns={'words': 'tokens'})

    splited_data = np.array_split(dataset.to_numpy(), CHUNKS_AMOUNT)

    dataset_results = []

    for i, chunk in enumerate(splited_data):
       print(f"Trabajando con chunk {i}")
       session_id = session_info[dataset_name][str(i)]

       if session_id is None: continue

       results = PubtatorAPI.get_results(session_id)

       chunk_df = pd.DataFrame(chunk, columns=['sentence_id', 'tokens', 'labels'])
       response_data = read_pubtator_content(results, chunk_df)

       entity_map = dataset_info['entity_map']

       metrics = calculate_metrics(chunk_df, response_data, entity_map, i)

       print(metrics)

       dataset_results.append(metrics)

    results = pd.DataFrame(dataset_results)

    results = results.drop(columns=['classification_report'])

    final_results[dataset_name] = results.mean().to_dict()